<!-- -->

In [ ]:
#| echo: false

import datetime
from IPython.display import display, Markdown
from tabulate import tabulate
import xarray as xr
from cartopy import crs
import holoviews as hv
import hvplot.pandas
from holoviews.element.tiles import tile_sources as ts
from holoviews import opts
from holoviews.operation.datashader import (
    rasterize, datashade, inspect_polygons
)
from holoviews.operation import timeseries

import colorcet as cc
import datashader as ds
import datashader.transfer_functions as tf

from bokeh.models import FixedTicker, HoverTool, DatetimeTickFormatter
from bokeh.models.formatters import DatetimeTickFormatter


from pyproj import Proj
from pyproj import Transformer

# warning extension('matplotlib') fait bugger l'affichage des plots geopandas.plot()
# tout le code est  fait pour être compatible avec bokeh.
hv.extension('bokeh')
# default value Analyses effectuées sur les données de l'année en cours si execute parameter n'est pas utilisé dans le lancement de la génération du rapport : --execute-params _variables.yml

today = datetime.date.today()

#Annee = today.year
Annee = 2020


export_file_path = './spatial-analyse_files/holoviz/'

In [ ]:
#| echo: false
#| output: false


## taille des plots fixés notament pour que les output png ne soit pas tronqué lors de l'utilisation de datashader

plot_width = 1200
plot_height = int(plot_width * 0.5)

plot_width_col2 = int(plot_width/2)
plot_height_col2 = int(plot_width_col2 * 0.5)

opts.defaults(
    opts.Polygons(
        xaxis=None, 
        yaxis=None,
    ),

    opts.Overlay( 
 


        #responsive=True
        )
    )
tiled_basemap = ts["CartoLight"]().opts(
    min_height=500, 
    #responsive=True, 
    xaxis=None, 
    yaxis=None)

In [ ]:
#| echo: false
#| output: false


styles = {'fontsize': 8}

texte_sources = "Sources: //////"
realisation = "Réalisation: //////"


# Coordonnées géographiques de la Nouvelle-Calédonie
lon, lat = 163.75, -22.4
lonN, latN = 167.1, -19.60
# Convertir en coordonnées Google Mercator (EPSG:3857)

transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
x, y = transformer.transform(lon, lat)
xN, yN = transformer.transform(lonN, latN)




def update_texte(x,y,texte, fontsize, halign):
    return hv.Text(x, y, texte, fontsize = fontsize,halign=halign)

def update_source(x,y,texte):
    x = x-50000
    y = y-50000
    return update_texte(x, y, f'Sources : {texte}', 8,'left' )

def update_realisation(x,y,texte):
    x = x-50000
    y = y-70000
    return update_texte(x, y, f'Réalisation : {texte}', 8,'left' )

# Positionnement du texte des sources
annotation_sources = update_source(x,y, texte_sources)

annotation_realisation = update_realisation(x,y, realisation)


# Création de la flèche
fleche_nord = hv.Arrow(xN, yN, 'N', '^').opts(
    text_color='black', 
    text_font_size= '10pt', 
    text_align='center', 
    text_baseline='middle', 
    text_font_style='bold', 
    text_alpha=0.5, 
    arrow_size=10, 
    arrow_color='black',
    arrow_alpha=0.5)    


# Définir les coordonnées de la barre d'échelle en coordonnées Google Mercator
# Ajustez ces valeurs en fonction de la taille souhaitée de votre barre d'échelle
x0, y0, x1, y1 = x-50000, y-500, x+50000, y+500 # 5000 mètres pour l'exemple

# Création de la barre d'échelle
echelle_barre = hv.Rectangles([(x0, y0, x1, y1)])
# Création de annotation d'échelle
texte_echelle = hv.Text(x, y-15000, '100 km')
